# Quickstart

To best understand how NutritionAI can give your agents super food-nutrition powers, let's build an agent that can find that information via Passio NutritionAI.

## Define tools

We first need to create [the Passio NutritionAI tool](/docs/integrations/tools/passio_nutritionai).

### [Passio Nutrition AI](/docs/integrations/tools/passio_nutritionai-agent)

We have a built-in tool in LangChain to easily use Passio NutritionAI to find food nutrition facts.
Note that this requires an API key - they have a free tier.

Once you create your API key, you will need to export that as:

```bash
export NUTRITIONAI_SUBSCRIPTION_KEY="..."
```

... or provide it to your Python environment via some other means such as the `dotenv` package.  You an also explicitly control the key via constructor calls.

In [2]:
from langchain_core.utils import get_from_env
from dotenv import load_dotenv
load_dotenv()

nutritionai_subscription_key = get_from_env("nutritionai_subscription_key", "NUTRITIONAI_SUBSCRIPTION_KEY")

In [3]:
# TODO: FOR PR TESTING ONLY
import sys
from pathlib import Path
langchain_git = Path().cwd().parents[3]
sys.path = [str(langchain_git / "libs" / "community"), str(langchain_git / "libs")] + sys.path

In [4]:
from langchain_community.tools.passio_nutritionai import NutritionAI
from langchain_community.utilities.passio_nutritionai import NutritionAIAPI

In [5]:
nutritionai_search = NutritionAI(api_wrapper=NutritionAIAPI())

In [6]:
nutritionai_search.invoke("chicken tikka masala")

{'results': [{'type': 'reference',
   'displayName': 'Chicken Tikka Masala',
   'stemmedDisplayName': '',
   'shortName': '',
   'longName': 'Chicken Tikka Masala',
   'scoredName': 'Chicken Tikka Masala',
   'score': 1,
   'displayNameScore': 1,
   'brandName': '',
   'iconId': '',
   'labelId': '',
   'synonymId': '',
   'recipeId': '',
   'referenceId': 'openfood0085239308349',
   'resultId': 'openfood0085239308349',
   'nutritionPreview': {'portion': {'weight': {'unit': 'g', 'value': 100},
     'name': '',
     'quantity': 1,
     'suggestedQuantity': [1]},
    'calories': 142.8571014404297}},
  {'type': 'reference',
   'displayName': 'Chicken Tikka Masala',
   'stemmedDisplayName': '',
   'shortName': '',
   'longName': 'Chicken Tikka Masala',
   'scoredName': 'Chicken Tikka Masala',
   'score': 0.95,
   'displayNameScore': 0.95,
   'brandName': 'Target Stores',
   'iconId': '',
   'labelId': '',
   'synonymId': '',
   'recipeId': '',
   'referenceId': '1603211257427',
   'resultI

### Tools

Now that we have the tool, we can create a list of tools that we will use downstream.

In [7]:
tools = [nutritionai_search]

## Create the agent

Now that we have defined the tools, we can create the agent. We will be using an OpenAI Functions agent - for more information on this type of agent, as well as other options, see [this guide](./agent_types)

First, we choose the LLM we want to be guiding the agent.

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

Next, we choose the prompt we want to use to guide the agent.

In [9]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

Now, we can initalize the agent with the LLM, the prompt, and the tools. The agent is responsible for taking in input and deciding what actions to take. Crucially, the Agent does not execute those actions - that is done by the AgentExecutor (next step). For more information about how to think about these components, see our [conceptual guide](./concepts)

In [10]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

Finally, we combine the agent (the brains) with the tools inside the AgentExecutor (which will repeatedly call the agent and execute tools). For more information about how to think about these components, see our [conceptual guide](./concepts)

In [11]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## Run the agent

We can now run the agent on a few queries! Note that for now, these are all **stateless** queries (it won't remember previous interactions).

In [12]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [14]:
agent_executor.invoke({"input": "how many calories are in a slice pepperoni pizza?"})



> Entering new AgentExecutor chain...

Invoking: `nutritionai_advanced_search` with `{'query': 'slice of pepperoni pizza'}`


{'results': [{'type': 'synonym', 'displayName': 'Fast Food, Pizza Chain, 14" Pizza, Pepperoni Topping, Regular Crust', 'stemmedDisplayName': '', 'shortName': 'slice of pepperoni pizza', 'longName': 'Fast Food, Pizza Chain, 14" Pizza, Pepperoni Topping, Regular Crust', 'scoredName': 'Slice Of Pepperoni Pizza', 'score': 1, 'displayNameScore': 0.6534380115251439, 'brandName': '', 'iconId': '1003594', 'labelId': '1f1bf95c-3256-11ed-920f-7e029295fe1f', 'synonymId': '1f1bf95f-3256-11ed-920f-7e029295fe1f', 'recipeId': '', 'referenceId': '1603211199398', 'resultId': '1603211199398', 'nutritionPreview': {'portion': {'weight': {'unit': 'g', 'value': 111}, 'name': 'slice', 'quantity': 1}, 'calories': 313.0199890136719}}, {'type': 'synonym', 'displayName': 'Domino\'s 14" Pepperoni Pizza, Classic Hand-Tossed Crust', 'stemmedDisplayName': '', 'shortName': 'slice of pepperon

{'input': 'how many calories are in a slice pepperoni pizza?',
 'output': 'A slice of pepperoni pizza typically contains around 260-350 calories, depending on the specific brand and crust type.'}

If we want to keep track of these messages automatically, we can wrap this in a RunnableWithMessageHistory. For more information on how to use this, see [this guide](/docs/expression_language/how_to/message_history)

In [15]:
agent_executor.invoke({"input": "I had bacon and eggs for breakfast.  How many calories is that?"})



> Entering new AgentExecutor chain...

Invoking: `nutritionai_advanced_search` with `{'query': 'bacon and eggs'}`


{'results': [{'type': 'recipe', 'displayName': 'English Muffin With Egg, Bacon And Cheese', 'stemmedDisplayName': '', 'shortName': 'English Muffin with Egg, Bacon and Cheese', 'longName': '', 'scoredName': 'Bacon And Egg Mcmuffin', 'score': 0.9336296056884292, 'displayNameScore': 0.7684418158923452, 'brandName': '', 'iconId': 'PRE0398', 'labelId': 'fccdbf14-cc85-11ea-965a-ab82c246b581', 'synonymId': '75da9f7e-7dd6-11eb-8ccb-c3fcc89180a9', 'recipeId': '68d57155-da31-11ec-b94d-ded9c6f70628', 'referenceId': '', 'resultId': '68d57155-da31-11ec-b94d-ded9c6f70628', 'nutritionPreview': {'portion': {'weight': {'unit': 'g', 'value': 148.1}, 'name': 'serving', 'quantity': 1, 'suggestedQuantity': [1]}, 'calories': 331.40802001953125}}, {'type': 'recipe', 'displayName': 'Puff Pastry Filled With Bacon And Egg', 'stemmedDisplayName': '', 'shortName': 'puff pastry filled with bacon an

{'input': 'I had bacon and eggs for breakfast.  How many calories is that?',
 'output': 'The bacon and eggs you had for breakfast have approximately 331 calories.'}

In [16]:
agent_executor.invoke({"input": "I had chicken tikka masala for lunch.  How much sodium did I have?"})



> Entering new AgentExecutor chain...

Invoking: `nutritionai_advanced_search` with `{'query': 'chicken tikka masala'}`


{'results': [{'type': 'reference', 'displayName': 'Chicken Tikka Masala', 'stemmedDisplayName': '', 'shortName': '', 'longName': 'Chicken Tikka Masala', 'scoredName': 'Chicken Tikka Masala', 'score': 1, 'displayNameScore': 1, 'brandName': '', 'iconId': '', 'labelId': '', 'synonymId': '', 'recipeId': '', 'referenceId': 'openfood0085239308349', 'resultId': 'openfood0085239308349', 'nutritionPreview': {'portion': {'weight': {'unit': 'g', 'value': 100}, 'name': '', 'quantity': 1, 'suggestedQuantity': [1]}, 'calories': 142.8571014404297}}, {'type': 'reference', 'displayName': 'Chicken Tikka Masala', 'stemmedDisplayName': '', 'shortName': '', 'longName': 'Chicken Tikka Masala', 'scoredName': 'Chicken Tikka Masala', 'score': 0.95, 'displayNameScore': 0.95, 'brandName': 'Target Stores', 'iconId': '', 'labelId': '', 'synonymId': '', 'recipeId': '', 'referenceId': '16032112

{'input': 'I had chicken tikka masala for lunch.  How much sodium did I have?',
 'output': "I found nutrition information for Chicken Tikka Masala. The calories per 100g serving are approximately 142.86. Unfortunately, I couldn't find the exact sodium content for the dish. If you have the nutrition label or specific brand information, I can help you calculate the sodium content."}

In [ ]:
agent_executor.invoke({"input": "I had sliced pepper jack cheese for a snack.  How much protein did I have?"})

## Conclusion

That's a wrap! In this quick start we covered how to create a simple agent that is able to incorporate food-nutrition information into its answers. Agents are a complex topic, and there's lot to learn! Head back to the [quick start](../../modules/agents/quick_start) or [main agent page](./) to find more resources on conceptual guides, different types of agents, how to create custom tools, and more!